# Data Ingestion to MongoDB

In [2]:
from pymongo import MongoClient
import math
import time

## Test Vector Search

In [4]:
#TODO: Sensitive Data, Replace password
CONNECTION_STRING = "mongodb+srv://aiinaction:<password>@starteconcluster1.cg9gxnb.mongodb.net/?retryWrites=true&w=majority&appName=StarteconCluster1"

In [5]:
client = MongoClient(CONNECTION_STRING)

### Get Collection

In [7]:
databases = client.list_database_names()
databases

['EcoWind', 'sample_mflix', 'sample_restaurants', 'admin', 'local']

In [8]:
db = client[databases[0]]

In [9]:
db.list_collection_names()

['wind_power_generation_data_05042017_competition',
 'wind_power_generation_data_04042017_competition',
 'meteorological_data_04042017_competition',
 'wind_power_generation_data_07042017_competition',
 'wind_power_generation_data_15042017_competition',
 'wind_power_generation_data_10042017_competition',
 'meteorological_data_10042017_competition',
 'meteorological_data_14042017_competition',
 'meteorological_data_12042017_competition',
 'wind_power_generation_data_08042017_competition',
 'wind_power_generation_data_06042017_competition',
 'meteorological_data_09042017_competition',
 'wind_power_generation_data_03042017_competition',
 'measurements',
 'observations_wind_power_vectors',
 'meteorological_data_13042017_competition',
 'meteorological_data',
 'meteorological_data_15042017_competition',
 'wind_power_generation_data_13042017_competition',
 'meteorological_data_16042017_competition',
 'meteorological_data_07042017_competition',
 'wind_power_generation_data_11042017_competition'

In [10]:
coll = db["observations_wind_power_vectors"]

### Get Query Vector

In [41]:
# First item is at day 2 at 00:00 o'clock
# We could also use id = 1
query_item = coll.find_one({"date": "002:00:00"})

In [13]:
type(query_item)

dict

In [14]:
vector = query_item["vectors"]

In [15]:
type(vector[0])

float

In [16]:
# define query pipeline
pipeline = [
  {
    '$vectorSearch': {
      'index': 'wind_power_vector_index', 
      'path': 'vectors', 
      'queryVector': vector, 
      'numCandidates': 100, 
      'limit': 10
    }
  }, {
    '$project': {
      '_id': 0, 
      'id': 1, 
      'date': 1, 
      'windpower': 1, 
      'score': {
        '$meta': 'vectorSearchScore'
      }
    }
  }
]

In [97]:
# run the pipeline
result = coll.aggregate(pipeline)

In [99]:
conf_high = []
conf_low = []
best = []

idx = 0
for i in result:
    if (i["id"] == query_item["id"]):
        continue
    if (idx == 1):
        best = i["windpower"]
        conf_high = i["windpower"]
        conf_low = i["windpower"]
        
    conf_high = list(map(max, zip(i["windpower"], conf_high)))
    conf_low = list(map(min, zip(i["windpower"], conf_low)))
    idx = idx + 1

In [101]:
x = 1
for y in best:
    print('{"x":%s, "y": %s}' % (x, y))
    x = x + 1

{"x":1, "y": 96.39}
{"x":2, "y": 98.97}
{"x":3, "y": 100.66}
{"x":4, "y": 107.67}
{"x":5, "y": 111.52}
{"x":6, "y": 100.66}
{"x":7, "y": 101.4}
{"x":8, "y": 92.91}
{"x":9, "y": 97.49}
{"x":10, "y": 93.07}
{"x":11, "y": 96.39}
{"x":12, "y": 90.43}
{"x":13, "y": 88.27}
{"x":14, "y": 92.17}
{"x":15, "y": 98.97}
{"x":16, "y": 102.14}
{"x":17, "y": 108.93}
{"x":18, "y": 113.21}
{"x":19, "y": 105.98}
{"x":20, "y": 101.92}
{"x":21, "y": 101.08}
{"x":22, "y": 93.43}
{"x":23, "y": 89.01}
{"x":24, "y": 90.33}
{"x":25, "y": 94.91}
{"x":26, "y": 86.27}
{"x":27, "y": 83.32}
{"x":28, "y": 81.21}
{"x":29, "y": 91.48}
{"x":30, "y": 89.7}
{"x":31, "y": 91.07}
{"x":32, "y": 101.55}
{"x":33, "y": 104.87}
{"x":34, "y": 110.3}
{"x":35, "y": 107.41}
{"x":36, "y": 109.52}
{"x":37, "y": 111.47}
{"x":38, "y": 114.84}
{"x":39, "y": 112.1}
{"x":40, "y": 122.22}
{"x":41, "y": 131.07}
{"x":42, "y": 130.23}
{"x":43, "y": 122.22}
{"x":44, "y": 129.44}
{"x":45, "y": 128.38}
{"x":46, "y": 125.12}
{"x":47, "y": 134.24}

In [103]:
x = 1
for y in conf_high:
    print('{"x":%s, "y": %s}' % (x, y))
    x = x + 1

{"x":1, "y": 201.92}
{"x":2, "y": 200.77}
{"x":3, "y": 198.55}
{"x":4, "y": 199.66}
{"x":5, "y": 197.71}
{"x":6, "y": 202.18}
{"x":7, "y": 201.18}
{"x":8, "y": 199.34}
{"x":9, "y": 199.34}
{"x":10, "y": 199.71}
{"x":11, "y": 200.44}
{"x":12, "y": 199.6}
{"x":13, "y": 203.24}
{"x":14, "y": 213.73}
{"x":15, "y": 227.27}
{"x":16, "y": 220.32}
{"x":17, "y": 210.99}
{"x":18, "y": 217.79}
{"x":19, "y": 202.66}
{"x":20, "y": 210.67}
{"x":21, "y": 209.67}
{"x":22, "y": 207.09}
{"x":23, "y": 206.72}
{"x":24, "y": 203.72}
{"x":25, "y": 203.72}
{"x":26, "y": 201.4}
{"x":27, "y": 203.77}
{"x":28, "y": 203.77}
{"x":29, "y": 204.87}
{"x":30, "y": 203.98}
{"x":31, "y": 204.5}
{"x":32, "y": 203.66}
{"x":33, "y": 201.08}
{"x":34, "y": 203.03}
{"x":35, "y": 204.03}
{"x":36, "y": 203.29}
{"x":37, "y": 203.24}
{"x":38, "y": 213.73}
{"x":39, "y": 227.27}
{"x":40, "y": 220.32}
{"x":41, "y": 210.99}
{"x":42, "y": 217.79}
{"x":43, "y": 202.5}
{"x":44, "y": 210.67}
{"x":45, "y": 209.67}
{"x":46, "y": 204.4}
{"

In [105]:
x = 1
for y in conf_low:
    print('{"x":%s, "y": %s}' % (x, y))
    x = x + 1

{"x":1, "y": 38.77}
{"x":2, "y": 44.62}
{"x":3, "y": 44.57}
{"x":4, "y": 42.73}
{"x":5, "y": 42.73}
{"x":6, "y": 40.51}
{"x":7, "y": 38.67}
{"x":8, "y": 38.72}
{"x":9, "y": 28.44}
{"x":10, "y": 26.12}
{"x":11, "y": 26.65}
{"x":12, "y": 24.8}
{"x":13, "y": 18.48}
{"x":14, "y": 13.68}
{"x":15, "y": 12.99}
{"x":16, "y": 15.47}
{"x":17, "y": 16.31}
{"x":18, "y": 20.0}
{"x":19, "y": 21.74}
{"x":20, "y": 21.48}
{"x":21, "y": 23.59}
{"x":22, "y": 23.54}
{"x":23, "y": 25.17}
{"x":24, "y": 26.91}
{"x":25, "y": 24.38}
{"x":26, "y": 25.01}
{"x":27, "y": 28.81}
{"x":28, "y": 24.75}
{"x":29, "y": 23.69}
{"x":30, "y": 20.96}
{"x":31, "y": 22.11}
{"x":32, "y": 17.27}
{"x":33, "y": 14.68}
{"x":34, "y": 13.31}
{"x":35, "y": 16.16}
{"x":36, "y": 11.47}
{"x":37, "y": 11.73}
{"x":38, "y": 11.15}
{"x":39, "y": 9.52}
{"x":40, "y": 7.41}
{"x":41, "y": 4.03}
{"x":42, "y": 3.03}
{"x":43, "y": 1.03}
{"x":44, "y": 2.87}
{"x":45, "y": 1.87}
{"x":46, "y": 2.87}
{"x":47, "y": 1.87}
{"x":48, "y": 5.24}


In [ ]:
#TODO: Convert High and Low to confidence interval